# OpenLangMem

In [13]:
# if you need it
# %pip install anthropic

In [10]:
import anthropic
import json
import uuid
import getpass
import os


os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("ANTHROPIC_API_KEY")

client = Anthropic()
MODEL_NAME = "claude-3-haiku-20240307"

johnny_user_id = uuid.uuid4()
jimmy_user_id = uuid.uuid4()
jimmy_username = f"jimmy-{uuid.uuid4().hex[:4]}"
johnny_username = f"johnny-{uuid.uuid4().hex[:4]}"

ANTHROPIC_API_KEY ········


In [11]:
# Unique for a given converstaion
thread_id = uuid.uuid4()

messages = [
    {"role": "system", "content": "You are a helpful AI assistant"},
    {
        "role": "user",
        # Names are optional but should be consistent with a given user id, if provided
        "name": jimmy_username,
        "content": "Hey johnny have i ever told you about my older bro steve?",
        "metadata": {
            "user_id": str(jimmy_user_id),
        },
    },
    {
        "content": "no, you didn't, but I think he was friends with my younger sister sueann",
        "role": "user",
        "name": johnny_username,
        "metadata": {
            "user_id": str(johnny_user_id),
        },
    },
    {
        "content": "yeah me and him used to play stickball down in the park before school started. I think it was in 1980",
        "role": "user",
        "name": jimmy_username,
        "metadata": {
            "user_id": str(jimmy_user_id),
        },
    },
    {
        "content": "That was totally 1979! I remember because i was stuck at home all summer.",
        "role": "user",
        "name": "Jeanne",
        # If the user ID isn't provided, we treat this as a guest message and won't assign memories to the user
    },
    {
        "content": "That was so long ago. I have gotten old and gained 200 pounds since then. I can't even remember who was president. @ai, who was the president in 1980?",
        "role": "user",
        "name": johnny_username,
        "metadata": {
            "user_id": str(johnny_user_id),
        },
    },
    {
        "content": "The president of the United States in 1980 was Jimmy Carter.",
        "role": "assistant",
    },
    {
        "content": "Wow ya i forgot that. Stickleball really impacted my life. It's how i first met Jeanne! wonder how my life would have turned out if it hadn't happened that way.",
        "role": "user",
        "name": jimmy_username,
        "metadata": {
            "user_id": str(jimmy_user_id),
        },
    },
    {
        "content": "Yeah wow. That was a big year! @ai could you remind me what else was going on that year?",
        "role": "user",
        "name": johnny_username,
        "metadata": {
            "user_id": str(johnny_user_id),
        },
    },
]

# result = await completion(messages)

# messages.append(result.choices[0].message)
# print(result.choices[0].message.content)

# Thread Summary

In [12]:
tools = [
    {
        "name": "thread_summary",
        "description": "Prints a summary of the conversation.",
        "input_schema": {
            "type": "object",
            "properties": {
                "title": {"type": "string", "description": "Distinct for the conversation."},
                "topics": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": 'Array of tags for topics discussed in this conversation, e.g. ["tech", "politics"]. Should be as specific as possible, and can overlap.'
                },
                "summary": {"type": "string", "description": "High level summary of the interactions. One or two paragraphs max."},
                "coherence": {"type": "integer", "description": "Coherence of the conversation, 0-100 (inclusive)"},
                "persuasion": {"type": "number", "description": "Conversation's persuasion score, 0.0-1.0 (inclusive)"}
            },
            "required": ['title', 'topics', 'summary', 'coherence', 'persuasion', 'counterpoint']
        }
    }
]

def create_pretty_string(msgs):
    result_lines = []
    for msg in msgs:
        name = msg.get('name', 'Unknown User') if msg.get('role') == 'user' else 'system'
        result_lines.append(f"{name}: {msg['content']}")
    return "\n".join(result_lines)

query = f"""
<conversation>
{create_pretty_string(messages)}
</conversation>

Use the `thread_summary` tool.
"""

response = client.beta.tools.messages.create(
    model=MODEL_NAME,
    max_tokens=4096,
    tools=tools,
    messages=[{"role": "user", "content": query}]
)
json_summary = None
for content in response.content:
    if content.type == "tool_use" and content.name == "thread_summary":
        json_summary = content.input
        break

if json_summary:
    print("JSON Summary:")
    print(json.dumps(json_summary, indent=2))
else:
    print("No JSON summary found in the response.")

JSON Summary:
{
  "title": "Old Childhood Memories",
  "topics": [
    "childhood",
    "sports",
    "relationships"
  ],
  "summary": "The conversation revolves around the nostalgic memories of the characters, particularly Jimmy's older brother Steve and their shared experience of playing stickball in the park before school in 1980. This childhood sport activity seems to have had a significant impact on Jimmy's life, as it was how he first met Jeanne. The characters fondly recall this time period and wonder how their lives would have turned out differently if that activity had not occurred. Overall, the discussion centers on the formative experiences and relationships from the characters' younger days.",
  "coherence": 80,
  "persuasion": 0.7
}


# User Profile

In [16]:
tools = [
    {
        "name": "user_state",
        "description": "Prints extract named entities.",
        "input_schema": {
            "type": "object",
            "properties": {
                "preferred_name": {"type": "string", "description": "The user's name."},
                "summary": {"type": "string", "description": "A quick summary of how the user would describe themselves."},
                "interests": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": 'Array of short (two to three word) descriptions of areas of particular interest for the user. This can be a concept, activity, or idea. Favor broad interests over specific ones.'
                },
                "other_info": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": ''
                },
                "relationships": {
                    "type": "array",
                    "description": 'A list of friends, family members, colleagues, and other relationships.',
                    "items": {
                        "type": "object",
                        "description": "A person's biographical details.",
                        "properties": {
                            "name": {"type": "string", "description": "The name of the person."},
                            "relation": {"type": "string", "description": "The relation of the person to the user."},
                            "context": {"type": "string", "description": "A detailed yet concise history of things the person has done with the user."}
                        },
                        "required": ["name", "relation", "context"]
                    }
                }
            },
            "required": ["summary"]
        }
    }
]

text = "John works at Google in New York. He met with Sarah, the CEO of Acme Inc., last week in San Francisco."

query = f"""
<document>
{text}
</document>

Use the user_state tool.
"""

response = client.beta.tools.messages.create(
    model=MODEL_NAME,
    max_tokens=4096,
    tools=tools,
    messages=[{"role": "user", "content": query}]
)

json_entities = None
for content in response.content:
    if content.type == "tool_use" and content.name == "user_state":
        json_entities = content.input
        break

if json_entities:
    print("Extracted Entities (JSON):")
    print(json_entities)
else:
    print("No entities found in the response.")

Extracted Entities (JSON):
{'interests': ['technology', 'business', 'travel'], 'other_info': ['works at Google', 'met with Sarah the CEO of Acme Inc.', 'in New York', 'in San Francisco'], 'preferred_name': 'John', 'relationships': [{'name': 'Sarah', 'relation': 'CEO of Acme Inc.', 'context': 'met with her last week in San Francisco'}], 'summary': 'John works in the technology industry, is interested in business, and has traveled to San Francisco and New York for work.'}
